# 多変量線形重回帰モデル

## 目的

世帯に関するあるデータを入力した際、出力としてそれっぽい緯度と経度を得る。  
使用例: どこに住もうかな？


## 結論

変数の相関があまりにもなく、まともなモデルを作成できなかった。


## 目次
1. 従属変数が複数ある場合の手法
1. データを世帯ごとに直す
1. データクレンジング
1. モデルの作成と評価
1. 独立変数を評価


## 従属変数が複数ある場合の手法

多変量線形重回帰モデルが使えそう。  

### 多変量線形重回帰モデルとは？

#### 回帰

連続している入力値から次の値を予測して、他の変数にどんな影響が与えられるか調査する手法(※1)

※1 https://gen-ai-media.guga.or.jp/glossary/regression/

#### 重回帰

重回帰とは、独立変数(入力)が複数ある回帰。  
独立変数が複数あるので、スカラーではなくベクトルとして扱う。  

※ 単回帰とは独立変数が1つ。  

#### 線形

以下の性質を満たす。  
f(x + y) = f(x) + f(y)  
f(kx) = kf(x)  

※y = w0 + w1x + w2x^2 の線分は曲線となるが、これも線形回帰問題と定義される。(※2)  
　このモデル式で解くべきw0, w1, w2といった重みが1次の項となり線形性を持つため。  

※2 人工知能プログラミングのための数学がわかる本 p145

#### 多変量

多変量とは、従属変数が複数ある回帰。  
独立変数が複数あるので、スカラーではなくベクトルとして扱う。  

https://user.keio.ac.jp/~nagakura/zemi/K_regression.pdf
p4
解き方 p6


## データを世帯ごとに直す

世帯に関するデータを扱いたい。

L36


## データクレンジング

最大値が明らかにおかしい。  
=> おかしい地点を調べてみると、刑務所だった。  
=> 今回の目的は「世帯に関するあるデータを入力した際、出力としてそれっぽい緯度と経度を得る」ことであるため、刑務所のデータは不要である。  
　　そのため、家庭っぽくないデータを消したい。  

そのため、以下の式を満たさないデータを消した。  
X = 平均 +- (標準偏差 * 2)  

※人口等、この手法ではまともに消せない項目もあるため、本来は各データの基本統計量やグラフを見ながら値ごとに調整する必要がある。  

以下を確認  
データクレンジング/生データ => 世帯ごとで集計  
データクレンジング/生データ => 世帯ごとで集計 => クレンジング  
メモ: 人口  


## モデルの作成と評価

メモ: 1つスキップ

In [33]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import itertools


# データ読み込み
cal_housing = pd.read_csv('datasets/cal_housing/CaliforniaHousing/cal_housing.data', sep=',')
cal_housing.columns = ['longitude', 'latitude', 'housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue']


# データクレンジング
def cleansing(df, col, n):
  mean = df[col].mean()
  std = df[col].std()

  df = df[(df[col] > mean - n * std) & (df[col] < mean + n * std)]
  
  return df

cal_housing = cleansing(cal_housing, 'housingMedianAge', 2)
cal_housing = cleansing(cal_housing, 'totalRooms', 2)
cal_housing = cleansing(cal_housing, 'totalBedrooms', 2)
cal_housing = cleansing(cal_housing, 'population', 2)
cal_housing = cleansing(cal_housing, 'households', 2)
cal_housing = cleansing(cal_housing, 'medianIncome', 2)
cal_housing = cleansing(cal_housing, 'medianHouseValue', 2)

# 世帯ごとで集計
cal_housing['population'] = cal_housing['population'] / cal_housing['households']
cal_housing['totalRooms'] = cal_housing['totalRooms'] / cal_housing['households']
cal_housing['totalBedrooms'] = cal_housing['totalBedrooms'] / cal_housing['households']


# 関連する基本統計量
display(cal_housing.describe())

def train(X, y):
  # 訓練データとテストデータに分割
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

  # 学習
  model = LinearRegression()
  model.fit(X_train, y_train)

  # 評価
  mse = mean_squared_error(y_test, model.predict(X_test))
  r2 = r2_score(y_test, model.predict(X_test))

  return model, r2


# 学習・評価
use_cal_housing = cal_housing[['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue']]
max_r2 = 0
max_combination = ''
for r in range(1, len(use_cal_housing) + 1):
  for combination in itertools.combinations(use_cal_housing, r):
    X = use_cal_housing[list(combination)]
    longitude_model, r2 = train(X, cal_housing[['longitude', 'latitude']])
    if r2 > max_r2:
      max_r2 = r2
      max_combination = combination

print(f"Best combination: {max_combination}")
print(f"Best R2: {max_r2}")

#X = cal_housing[['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue']]
#longitude_model = train(X, cal_housing[['longitude', 'latitude']])



,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000
mean,-119.611961,35.729048,29.915836,5.313109,1.089453,2.974429,384.378195,3.527576,178709.624041
std,2.004892,2.184998,12.069361,1.953009,0.359071,1.005249,167.153233,1.404762,84774.564866
min,-124.350000,32.540000,4.000000,0.888889,0.500000,0.692308,20.000000,0.499900,14999.000000
25%,-121.760000,33.940000,20.000000,4.449187,1.000000,2.476526,264.000000,2.455400,110400.000000
50%,-118.770000,34.420000,31.000000,5.177542,1.045808,2.856337,370.000000,3.358150,165300.000000
75%,-118.030000,37.750000,38.000000,5.915544,1.099243,3.316456,499.000000,4.430750,233725.000000
max,-114.490000,41.950000,52.000000,62.422222,14.111111,63.750000,779.000000,7.723400,405400.000000


Best combination: ('housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue')
Best R2: 0.08708392536428161


## モデルの作成と評価

従属変数: 緯度・軽度  
独立変数: それ以外  

以下のように実行したところ、精度が8%...  

なぜ？...  


データクレンジング/生データ => 世帯ごとで集計 => クレンジング  

仮説  
以下の理由によりまともな回帰直線が引けないのではないか？  
- 相関グラフより、独立変数になんらかの係数をかけて足し合わせても、従属変数との相関がなさすぎるため。

解決策(仮説)
1. 独立変数の特徴が近いものがあると仮定して、これらをグループ化。  
　　=> k-means法
1. グループ化した際のラベルをエリアとして、返すようにしたい。  
　　=> 線形にならないので、別の方法を検討。  
　　NNとか

In [34]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import itertools


# データ読み込み
cal_housing = pd.read_csv('datasets/cal_housing/CaliforniaHousing/cal_housing.data', sep=',')
cal_housing.columns = ['longitude', 'latitude', 'housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue']


# データクレンジング
def cleansing(df, col, n):
  mean = df[col].mean()
  std = df[col].std()

  df = df[(df[col] > mean - n * std) & (df[col] < mean + n * std)]
  
  return df

cal_housing = cleansing(cal_housing, 'housingMedianAge', 2)
cal_housing = cleansing(cal_housing, 'totalRooms', 2)
cal_housing = cleansing(cal_housing, 'totalBedrooms', 2)
cal_housing = cleansing(cal_housing, 'population', 2)
cal_housing = cleansing(cal_housing, 'households', 2)
cal_housing = cleansing(cal_housing, 'medianIncome', 2)
cal_housing = cleansing(cal_housing, 'medianHouseValue', 2)

# 世帯ごとで集計
cal_housing['population'] = cal_housing['population'] / cal_housing['households']
cal_housing['totalRooms'] = cal_housing['totalRooms'] / cal_housing['households']
cal_housing['totalBedrooms'] = cal_housing['totalBedrooms'] / cal_housing['households']


# 関連する基本統計量
display(cal_housing.describe())

def train(X, y):
  # 訓練データとテストデータに分割
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

  # 学習
  model = LinearRegression()
  model.fit(X_train, y_train)

  # 評価
  mse = mean_squared_error(y_test, model.predict(X_test))
  r2 = r2_score(y_test, model.predict(X_test))
  print('COEF', model.coef_)
  print('INTERCEPT', model.intercept_)
  print('MSE:', mse)
  print('R2:', r2)

  return model, r2


# 学習・評価
X = cal_housing[['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue']]
longitude_model = train(X, cal_housing[['longitude', 'latitude']])



,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000
mean,-119.611961,35.729048,29.915836,5.313109,1.089453,2.974429,384.378195,3.527576,178709.624041
std,2.004892,2.184998,12.069361,1.953009,0.359071,1.005249,167.153233,1.404762,84774.564866
min,-124.350000,32.540000,4.000000,0.888889,0.500000,0.692308,20.000000,0.499900,14999.000000
25%,-121.760000,33.940000,20.000000,4.449187,1.000000,2.476526,264.000000,2.455400,110400.000000
50%,-118.770000,34.420000,31.000000,5.177542,1.045808,2.856337,370.000000,3.358150,165300.000000
75%,-118.030000,37.750000,38.000000,5.915544,1.099243,3.316456,499.000000,4.430750,233725.000000
max,-114.490000,41.950000,52.000000,62.422222,14.111111,63.750000,779.000000,7.723400,405400.000000


COEF [[-1.55383634e-02 -4.03979339e-01  1.91618980e+00  2.00768712e-01
   3.23904003e-04  2.90038922e-01 -1.30998423e-06]
 [ 3.76242484e-03  6.58373579e-01 -2.58517148e+00 -2.72063316e-01
  -5.33887878e-04 -3.89869414e-01 -3.72345702e-06]]
INTERCEPT [-120.6178064    38.01487006]
MSE: 3.984467748313468
R2: 0.08304292700042981


## クラスタリング-多変量重回帰

In [35]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.cluster import KMeans


# データ読み込み
cal_housing = pd.read_csv('datasets/cal_housing/CaliforniaHousing/cal_housing.data', sep=',')
cal_housing.columns = ['longitude', 'latitude', 'housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue']


# データクレンジング
def cleansing(df, col, n):
  mean = df[col].mean()
  std = df[col].std()

  df = df[(df[col] > mean - n * std) & (df[col] < mean + n * std)]
  
  return df

cal_housing = cleansing(cal_housing, 'housingMedianAge', 2)
cal_housing = cleansing(cal_housing, 'totalRooms', 2)
cal_housing = cleansing(cal_housing, 'totalBedrooms', 2)
cal_housing = cleansing(cal_housing, 'population', 2)
cal_housing = cleansing(cal_housing, 'households', 2)
cal_housing = cleansing(cal_housing, 'medianIncome', 2)
cal_housing = cleansing(cal_housing, 'medianHouseValue', 2)

# 世帯ごとで集計
cal_housing['population'] = cal_housing['population'] / cal_housing['households']
cal_housing['totalRooms'] = cal_housing['totalRooms'] / cal_housing['households']
cal_housing['totalBedrooms'] = cal_housing['totalBedrooms'] / cal_housing['households']


# 関連する基本統計量
display(cal_housing.describe())

def train(X, y):
  # 訓練データとテストデータに分割
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

  # 学習
  model = LinearRegression()
  model.fit(X_train, y_train)

  # 評価
  mse = mean_squared_error(y_test, model.predict(X_test))
  r2 = r2_score(y_test, model.predict(X_test))
  # print('COEF', model.coef_)
  # print('INTERCEPT', model.intercept_)
  # print('MSE:', mse)
  # print('R2:', r2)

  return model, r2


# クラスタリング
r2_max = 0
n_max = 0
for n in range(10, 100):
  clf = KMeans(n_clusters=n, init='k-means++', n_init=10, max_iter=30000,tol=0.0001)
  pred = clf.fit_predict(cal_housing)
  cal_housing['cluster'] = pred

  # 学習・評価
  X = cal_housing[['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population', 'households', 'medianIncome', 'medianHouseValue']]
  longitude_model, r2 = train(X, cal_housing[['cluster']])

  if r2 > r2_max:
    r2_max = r2
    n_max = n

print(f"Best n: {n_max}")
print(f"Best R2: {r2_max}")



,longitude,latitude,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000,15648.000000
mean,-119.611961,35.729048,29.915836,5.313109,1.089453,2.974429,384.378195,3.527576,178709.624041
std,2.004892,2.184998,12.069361,1.953009,0.359071,1.005249,167.153233,1.404762,84774.564866
min,-124.350000,32.540000,4.000000,0.888889,0.500000,0.692308,20.000000,0.499900,14999.000000
25%,-121.760000,33.940000,20.000000,4.449187,1.000000,2.476526,264.000000,2.455400,110400.000000
50%,-118.770000,34.420000,31.000000,5.177542,1.045808,2.856337,370.000000,3.358150,165300.000000
75%,-118.030000,37.750000,38.000000,5.915544,1.099243,3.316456,499.000000,4.430750,233725.000000
max,-114.490000,41.950000,52.000000,62.422222,14.111111,63.750000,779.000000,7.723400,405400.000000


Best n: 13
Best R2: 0.07593594648385804
